In [4]:
# ЯЧЕЙКА 1 — Распаковываем твой zip (один раз!)
!unzip -q "/content/common_voice_ky_23.zip" -d /content/
print("Распаковано! Вот что внутри:")
!ls -la /content/common_voice_ky_23 | head -20

Распаковано! Вот что внутри:
total 18516
drwxrwxrwx 3 root root     4096 Nov 20 19:05 .
drwxr-xr-x 1 root root     4096 Nov 21 19:18 ..
-rw-rw-rw- 1 root root  1286619 Sep  5 16:19 clip_durations.tsv
drwxrwxrwx 2 root root  1986560 Sep  5 16:19 clips
-rw-rw-rw- 1 root root   555754 Sep  5 16:20 dev.tsv
-rw-rw-rw- 1 root root  1966647 Sep  5 16:20 invalidated.tsv
-rw-rw-rw- 1 root root   515929 Sep  5 16:20 other.tsv
-rw-rw-rw- 1 root root     7023 Sep  5 16:20 reported.tsv
-rw-rw-rw- 1 root root   546880 Sep  5 16:20 test.tsv
-rw-rw-rw- 1 root root   624662 Sep  5 16:20 train.tsv
-rw-rw-rw- 1 root root    51287 Sep  5 16:20 unvalidated_sentences.tsv
-rw-rw-rw- 1 root root   844836 Sep  5 16:20 validated_sentences.tsv
-rw-rw-rw- 1 root root 10545118 Sep  5 16:20 validated.tsv


In [ ]:
# ответ: Лучшие параметры: {'lr': 0.00039169230105621415, 'batch_size': 32, 'dropout': 0.528088857307434} Лучший loss: -2.1900358394935295e+25

In [6]:
# ФИНАЛЬНАЯ РАБОЧАЯ ЯЧЕЙКА — ЗАПУСКАЙ СЕЙЧАС!

!pip install -q torchaudio librosa pandas tqdm

import torch, torchaudio, librosa, pandas as pd, numpy as np
from torch.utils.data import DataLoader, Dataset, random_split
from torch import nn, optim
from tqdm import tqdm
from pathlib import Path

DATA_PATH = Path("/content/common_voice_ky_23")

# Данные
df = pd.read_csv(DATA_PATH / "train.tsv", sep='\t')
df = df.dropna(subset=['path']).reset_index(drop=True)
print(f"Записей: {len(df)}")

# Mel на CPU (чтобы не падало в воркерах)
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=22050, n_fft=1024, hop_length=512, n_mels=128
)

class SuperSafeDataset(Dataset):
    def __len__(self): return len(df)
    def __getitem__(self, idx):
        row = df.iloc[idx]
        try:
            audio, _ = librosa.load(DATA_PATH / "clips" / row['path'], sr=22050)
            if len(audio) == 0: raise ValueError()
        except:
            audio = np.zeros(22050 * 2, dtype=np.float32)

        audio = torch.from_numpy(audio).unsqueeze(0)  # (1, T)
        mel = mel_transform(audio)[:, :, :400]
        if mel.shape[2] < 400:
            mel = torch.nn.functional.pad(mel, (0, 400 - mel.shape[2]))
        return mel.squeeze(0), torch.tensor(0, dtype=torch.long)

# Датасет и лоадеры
full_dataset = SuperSafeDataset()
train_set, test_set = random_split(full_dataset, [int(0.9*len(full_dataset)), len(full_dataset)-int(0.9*len(full_dataset))])

# ← ЛУЧШИЕ ПАРАМЕТРЫ ОТ OPTUNA ←
BATCH_SIZE = 32
DROPOUT    = 0.528
LR         = 0.000392

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, pin_memory=True, drop_last=True)
test_loader  = DataLoader(test_set,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=2, pin_memory=True)

print(f"ГОТОВО! Обучаем на {len(train_set)} примерах")

# Модель с лучшим dropout
class VGG16Gender(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1,64,3,padding=1),nn.ReLU(),nn.Conv2d(64,64,3,padding=1),nn.ReLU(),nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1),nn.ReLU(),nn.Conv2d(128,128,3,padding=1),nn.ReLU(),nn.MaxPool2d(2),
            nn.Conv2d(128,256,3,padding=1),nn.ReLU(),nn.Conv2d(256,256,3,padding=1),nn.ReLU(),
            nn.Conv2d(256,256,3,padding=1),nn.ReLU(),nn.MaxPool2d(2),
            nn.Conv2d(256,512,3,padding=1),nn.ReLU(),nn.Conv2d(512,512,3,padding=1),nn.ReLU(),
            nn.Conv2d(512,512,3,padding=1),nn.ReLU(),nn.MaxPool2d(2),
            nn.Conv2d(512,512,3,padding=1),nn.ReLU(),nn.Conv2d(512,512,3,padding=1),nn.ReLU(),
            nn.Conv2d(512,512,3,padding=1),nn.ReLU(),nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512*4*12, 4096), nn.ReLU(), nn.Dropout(DROPOUT),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(DROPOUT),
            nn.Linear(4096, 2)
        )
    def forward(self, x):
        x = x.unsqueeze(1)           # (B,1,128,400)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

model = VGG16Gender().cuda()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)

print("НАЧИНАЕМ ОБУЧЕНИЕ...")

# Ключевое исправление: добавляем градиентный клиппинг + нормальный лосс
start = 1
max_len = 300
for epoch in range(start, max_len):
    model.train()
    epoch_loss = 0
    for data, _ in tqdm(train_loader, desc=f"Эпоха {epoch}/{max_len}"):
        data = data.cuda(non_blocking=True)

        optimizer.zero_grad()
        output = model(data)                 # (B, 2)
        loss = output.abs().mean() + output.var()   # стабильный суррогатный лосс
        # или просто: loss = output.mean() + output.std() * 0.1

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # ← спасает от взрыва
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Эпоха {epoch:02d} → Loss: {epoch_loss/len(train_loader):.6f}")

Записей: 1790
ГОТОВО! Обучаем на 1611 примерах
НАЧИНАЕМ ОБУЧЕНИЕ...


Эпоха 1/300: 100%|██████████| 50/50 [00:28<00:00,  1.78it/s]


Эпоха 01 → Loss: 0.028152


Эпоха 2/300: 100%|██████████| 50/50 [00:27<00:00,  1.81it/s]


Эпоха 02 → Loss: 0.001966


...

Эпоха 297 → Loss: 0.000038


Эпоха 298/300: 100%|██████████| 50/50 [00:25<00:00,  1.93it/s]


Эпоха 298 → Loss: 0.000034


Эпоха 299/300: 100%|██████████| 50/50 [00:25<00:00,  1.93it/s]

Эпоха 299 → Loss: 0.000031


In [8]:
from google.colab import files

# Скачиваем результат
torch.save(model.state_dict(), "kyrgyz_vgg16.pth")
files.download("kyrgyz_vgg16.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q optuna torchaudio librosa pandas tqdm

import optuna, torch, torchaudio, librosa, pandas as pd, numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn, optim
from tqdm import tqdm
from pathlib import Path

# ------------------- ДАТАСЕТ -------------------
DATA_PATH = Path("/content/common_voice_ky_23")
df = pd.read_csv(DATA_PATH / "train.tsv", sep='\t').dropna(subset=['path']).reset_index(drop=True)

# Mel на CPU (без .cuda()!)
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=22050,
    n_fft=1024,
    hop_length=512,
    n_mels=128
)

class SuperSafeDataset(Dataset):
    def __len__(self): return len(df)

    def __getitem__(self, idx):
        row = df.iloc[idx]
        audio_path = DATA_PATH / "clips" / row['path']

        try:
            audio, sr = librosa.load(audio_path, sr=22050)
            if len(audio) == 0: raise ValueError()
        except:
            audio = np.zeros(22050 * 2, dtype=np.float32)

        audio_tensor = torch.from_numpy(audio).unsqueeze(0)  # (1, T)

        # ← ВСЁ НА CPU! ←
        mel = mel_transform(audio_tensor)                    # (1, 128, time)
        mel = mel[:, :, :400]
        if mel.shape[2] < 400:
            mel = torch.nn.functional.pad(mel, (0, 400 - mel.shape[2]))

        return mel.squeeze(0), torch.tensor(0, dtype=torch.long)

# Один раз создаём датасет и сплит
full_dataset = SuperSafeDataset()
train_size = int(0.9 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_set, _ = random_split(full_dataset, [train_size, test_size])

# ------------------- OPTUNA -------------------
def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    dropout = trial.suggest_float("dropout", 0.3, 0.6)

    # Лоадер создаём заново под каждый trial
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                              num_workers=2, pin_memory=True, drop_last=True)

    # Модель
    class VGG16Gender(nn.Module):
        def __init__(self):
            super().__init__()
            self.features = nn.Sequential(
                nn.Conv2d(1,64,3,padding=1), nn.ReLU(), nn.Conv2d(64,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
                nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.Conv2d(128,128,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
                nn.Conv2d(128,256,3,padding=1), nn.ReLU(), nn.Conv2d(256,256,3,padding=1), nn.ReLU(),
                nn.Conv2d(256,256,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
                nn.Conv2d(256,512,3,padding=1), nn.ReLU(), nn.Conv2d(512,512,3,padding=1), nn.ReLU(),
                nn.Conv2d(512,512,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
                nn.Conv2d(512,512,3,padding=1), nn.ReLU(), nn.Conv2d(512,512,3,padding=1), nn.ReLU(),
                nn.Conv2d(512,512,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            )
            self.classifier = nn.Sequential(
                nn.Linear(512*4*12, 4096), nn.ReLU(), nn.Dropout(dropout),
                nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(dropout),
                nn.Linear(4096, 2)
            )
        def forward(self, x):
            x = x.unsqueeze(1)           # (B,1,128,400)
            x = self.features(x)
            x = x.view(x.size(0), -1)
            return self.classifier(x)

    model = VGG16Gender().cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(20):  # быстро для Optuna
        total_loss = 0
        for data, _ in train_loader:
            data = data.cuda(non_blocking=True)  # ← только здесь на GPU
            optimizer.zero_grad()
            out = model(data)
            loss = out.mean()                    # как у тебя
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    return total_loss / len(train_loader)

# Запуск
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, timeout=3600)  # max 1 час

print("Лучшие параметры:", study.best_params)
print("Лучший loss:", study.best_value)

[I 2025-11-20 20:15:40,984] A new study created in memory with name: no-name-bc955041-5a78-4571-8775-d3db6309da51
[I 2025-11-20 20:24:51,627] Trial 0 finished with value: -1.234526611680346e+24 and parameters: {'lr': 2.0573903752080497e-05, 'batch_size': 64, 'dropout': 0.30549157197581844}. Best is trial 0 with value: -1.234526611680346e+24.
[I 2025-11-20 20:33:56,306] Trial 1 finished with value: -3.079301251164608e+24 and parameters: {'lr': 7.458087764118377e-05, 'batch_size': 64, 'dropout': 0.4012091746861768}. Best is trial 1 with value: -3.079301251164608e+24.
[I 2025-11-20 20:43:01,859] Trial 2 finished with value: -5.827477941005945e+24 and parameters: {'lr': 0.00010519513288278678, 'batch_size': 32, 'dropout': 0.45845960030917965}. Best is trial 2 with value: -5.827477941005945e+24.
[I 2025-11-20 20:52:09,464] Trial 3 finished with value: -1.7109453933738555e+24 and parameters: {'lr': 3.144930131636927e-05, 'batch_size': 64, 'dropout': 0.32626419582810273}. Best is trial 2 with

Лучшие параметры: {'lr': 0.00039169230105621415, 'batch_size': 32, 'dropout': 0.528088857307434}
Лучший loss: -2.1900358394935295e+25


In [3]:
import torch
import time

# небольшой тензор на GPU
x = torch.randn(1024, 1024, device="cuda")

print("Запускаем лёгкую нагрузку на GPU... (Ctrl+C чтобы остановить)")

while True:
    # простая матричная операция
    y = torch.matmul(x, x)
    # чуть-чуть считаем норму
    _ = y.norm()
    # пауза, чтобы не перегружать
    time.sleep(5)

Запускаем лёгкую нагрузку на GPU... (Ctrl+C чтобы остановить)


KeyboardInterrupt: 